# Taller Dia 2

In [6]:
# Añade tu api key aquí
import os
os.environ["OPENAI_API_KEY"] = "***REMOVED***"

In [8]:
from tools import *

In [10]:
check_openai()

Todo está ok! :)


True

In [12]:
# Creamos el cliente para los siguientes ejercicios
import openai
client = openai.OpenAI()

In [14]:

def explain_chatbot2(code:str): 
    response = client.chat.completions.create(max_tokens=3000, model="gpt-3.5-turbo-1106", 
                                              messages=[ {"role": "system", "content": "Traduce al inglés"}, 
                                                        {"role": "user", "content": "Hola"}]) 
    return response

In [21]:
codigo=f"""
def explica_codigo(codigo):

    respuesta = client.completions.create(model="gpt-3.5-turbo-instruct", prompt=codigo)
    return respuesta

"""

response = explain_chatbot2(codigo)

In [32]:
print(response.choices[0].message.content)

El código proporcionado es una función de Python llamada `explica_codigo` que toma un argumento `codigo`. La función utiliza un cliente (llamado `client`) para realizar una llamada a un servicio de completado de texto, posiblemente relacionado con OpenAI's GPT-3.5, para obtener una respuesta basada en el código proporcionado como entrada. La respuesta generada por el servicio se devuelve como resultado de la función.

Es importante tener en cuenta que el cliente utilizado en el código (referenciado como `client`) debe haber sido previamente definido e inicializado con las credenciales adecuadas para acceder al servicio de completado de texto. Además, es probable que el código de conexión y autorización necesite estar presente antes de ejecutar esta función.

En resumen, la función `explica_codigo` parece estar destinada a proporcionar una explicación o información adicional basada en el código proporcionado como entrada, utilizando algún tipo de servicio de inteligencia artificial para

In [ ]:
def explain_chatbot(code:str):
    
    response = llm(max_tokens=3000, model="gpt-3.5-turbo-1106",
                                       messages=[
    {"role": "system",
     "content": "Explica el código de python que te da el usuario"},
    {"role": "user",
     "content": f"""
```python
{code}
```
"""}
])
    assert response.choices[0].finish_reason == "stop", "Faltan tokens, incrementa max_tokens o reduce el prompt"
    return response.choices[0].message.content

In [ ]:
import inspect
import openai
from functools import lru_cache

_cache = {}

def make_cache_key(kwargs):
    def convert_to_hashable(value):
        if isinstance(value, dict):
            return frozenset((k, convert_to_hashable(v)) for k, v in sorted(value.items()))
        elif isinstance(value, list):
            return tuple(convert_to_hashable(v) for v in value)
        elif isinstance(value, set):
            return frozenset(convert_to_hashable(v) for v in value)
        else:
            return value

    return frozenset((k, convert_to_hashable(v)) for k, v in sorted(kwargs.items()))

def llm(**kwargs):
    
    key = make_cache_key(kwargs)
    try:
        return _cache[key]
    except KeyError:
        _cache[key] = result = client.chat.completions.create(**kwargs)
        return result

@lru_cache(3000)
def get_embedding(text, model="text-embedding-3-large"):

    client = openai.OpenAI()
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding


def get_module_functions(module_name):
    module = __import__(module_name)
    functions = []

    for name, obj in inspect.getmembers(module):
        if inspect.isfunction(obj) and obj.__module__ == module_name:
            ds = inspect.getdoc(obj)
            function_info = {
                "name": name,
                "docstring": ds
            }
            if ds:
                functions.append(function_info)

    return functions